In [45]:
import sys
sys.path.insert(0, '../../backend/src')

import numpy as np
import pandas as pd
from datetime import datetime, timezone
from boto3.dynamodb.conditions import Key
from utils import s3, constants, utils, db, weather

In [46]:
jumpstart_hist, _ = weather.fetch_observations((datetime.now(timezone.utc) - pd.Timedelta(days=constants.MAX_HISTORY_REACHBACK_YEARS * 365)), constants.USGS_SITE, '01427510')
print(jumpstart_hist)

jumpstart_end_dt = jumpstart_hist.index.max()
print(f'jumpstart ends at {jumpstart_end_dt}')

                           airtemp  precip  snow  snowdepth  cloudcover
2020-09-12 03:00:00+00:00     53.8     0.0   0.0        0.0         4.2
2020-09-12 04:00:00+00:00     53.1     0.0   0.0        0.0        14.7
2020-09-12 05:00:00+00:00     50.7     0.0   0.0        0.0        34.4
2020-09-12 06:00:00+00:00     49.7     0.0   0.0        0.0        52.3
2020-09-12 07:00:00+00:00     48.9     0.0   0.0        0.0         2.2
...                            ...     ...   ...        ...         ...
2023-08-01 23:00:00+00:00     73.0     0.0   0.0        0.0        64.1
2023-08-02 00:00:00+00:00     69.6     0.0   0.0        0.0         0.0
2023-08-02 01:00:00+00:00     62.4     0.0   0.0        0.0        17.1
2023-08-02 02:00:00+00:00     60.1     0.0   0.0        0.0        17.1
2023-08-02 03:00:00+00:00     58.0     0.0   0.0        0.0         0.0

[25297 rows x 5 columns]
jumpstart ends at 2023-08-02 03:00:00+00:00


In [50]:
res = db.table.query(
  KeyConditionExpression=Key('usgs_site#type')
      .eq(f'{constants.USGS_SITE}#hist') & Key('timestamp').gt(int(jumpstart_end_dt.timestamp()))
)
print(res)
print(len(res['Items']))

{'Items': [{'snow': Decimal('0'), 'precip': Decimal('0'), 'snowdepth': Decimal('0'), 'cloudcover': Decimal('0'), 'usgs_site#type': '01427510#hist', 'timestamp': Decimal('1690948800'), 'streamflow': Decimal('1850'), 'airtemp': Decimal('55.3'), 'usgs_site': '01427510', 'watertemp': Decimal('67.64'), 'type': 'hist'}, {'snow': Decimal('0'), 'precip': Decimal('0'), 'snowdepth': Decimal('0'), 'cloudcover': Decimal('0'), 'usgs_site#type': '01427510#hist', 'timestamp': Decimal('1690952400'), 'streamflow': Decimal('1830'), 'airtemp': Decimal('54.3'), 'usgs_site': '01427510', 'watertemp': Decimal('66.56'), 'type': 'hist'}, {'snow': Decimal('0'), 'precip': Decimal('0'), 'snowdepth': Decimal('0'), 'cloudcover': Decimal('0'), 'usgs_site#type': '01427510#hist', 'timestamp': Decimal('1690956000'), 'streamflow': Decimal('1830'), 'airtemp': Decimal('53.4'), 'usgs_site': '01427510', 'watertemp': Decimal('65.84'), 'type': 'hist'}, {'snow': Decimal('0'), 'precip': Decimal('0'), 'snowdepth': Decimal('0'), 

In [51]:
pred = lambda i: i['timestamp'] > 1694163600 or ('origin' in i.keys() and i['origin'] > 1694163600)
res['Items'] = list(filter(pred, res['Items']))
print(len(res['Items']))

87


In [52]:
print(f'deleting {len(res["Items"])} items')

with db.table.batch_writer() as batch:
  for item in res['Items']:
    pk = item['usgs_site#type']
    sk = item['timestamp']

    batch.delete_item(
      Key={
        'usgs_site#type': pk,
        'timestamp': sk
      }
    )

deleting 87 items
